# Implementación en numpy

In [1]:
import os
import numpy as np
import pickle
# Perceptron con python: https://pyimagesearch.com/2021/05/06/implementing-the-perceptron-neural-network-with-python/#pyis-cta-modal
# Perceptron con PyTorch: https://www.geeksforgeeks.org/what-is-perceptron-the-simplest-artificial-neural-network/
# Perceptrones multicapa con PyTorch: https://github.com/rasbt/machine-learning-book/blob/main/ch11/ch11.ipynb
# Implementación Forward Propagation: https://www.geeksforgeeks.org/what-is-forward-propagation-in-neural-networks/
# Implementación Back Propagation: https://www.geeksforgeeks.org/backpropagation-in-neural-network/

In [2]:
# https://www.kaggle.com/code/annisin/classification-task
# Imagenes blanco y negro
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Split dataset into training and testing sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the pixel values to range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Imagenes a color RGB
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_cifar10(data_dir='cifar-10-batches-py'):
    # Cargar datos de entrenamiento
    data = []
    labels = []
    for i in range(1, 6):
        batch = unpickle(os.path.join(data_dir, f'data_batch_{i}'))
        data.append(batch[b'data'])
        labels.append(batch[b'labels'])

    data = np.concatenate(data)
    labels = np.concatenate(labels)

    data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    labels = np.eye(10)[labels]  # One-hot encoding

    # Cargar datos de prueba
    test_batch = unpickle(os.path.join(data_dir, 'test_batch'))
    test_data = test_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    test_labels = np.eye(10)[test_batch[b'labels']]

    return data, labels, test_data, test_labels

def load_cifar100(data_dir='cifar-100-python'):
    data = unpickle(os.path.join(data_dir, 'train'))
    x_train = data[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    y_train = np.eye(100)[data[b'fine_labels']]

    data = unpickle(os.path.join(data_dir, 'test'))
    x_test = data[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    y_test = np.eye(100)[data[b'fine_labels']]

    return x_train, y_train, x_test, y_test

In [ ]:
# Imagenes a color RGB
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_cifar10(data_dir='cifar-10-batches-py'):
    # Cargar datos de entrenamiento
    data = []
    labels = []
    for i in range(1, 6):
        batch = unpickle(os.path.join(data_dir, f'data_batch_{i}'))
        data.append(batch[b'data'])
        labels.append(batch[b'labels'])

    data = np.concatenate(data)
    labels = np.concatenate(labels)

    data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    labels = np.eye(10)[labels]  # One-hot encoding

    # Cargar datos de prueba
    test_batch = unpickle(os.path.join(data_dir, 'test_batch'))
    test_data = test_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    test_labels = np.eye(10)[test_batch[b'labels']]

    return data, labels, test_data, test_labels

def load_cifar100(data_dir='cifar-100-python'):
    data = unpickle(os.path.join(data_dir, 'train'))
    x_train = data[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    y_train = np.eye(100)[data[b'fine_labels']]

    data = unpickle(os.path.join(data_dir, 'test'))
    x_test = data[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) / 255.0
    y_test = np.eye(100)[data[b'fine_labels']]

    return x_train, y_train, x_test, y_test

¡Perfecto! Aquí tienes una guía paso a paso para implementar una red neuronal multicapa (MLP) para clasificación de imágenes, siguiendo la consigna y los recursos que mencionaste:

1. **Preparar los datos**
   - Descarga y carga los datasets (MNIST, CIFAR-10, CIFAR-100).
   - Preprocesa: normaliza los valores de píxeles y convierte las etiquetas a formato numérico o one-hot.

2. **Implementación en numpy (bajo nivel)**
   - Define la arquitectura: decide el número de capas ocultas y neuronas por capa.
   - Inicializa los pesos y sesgos aleatoriamente.
   - Implementa la propagación hacia adelante (forward propagation) usando funciones de activación (sigmoide, ReLU, etc.).
   - Implementa la retropropagación (backpropagation) para actualizar los pesos usando el gradiente descendente.
   - Entrena la red: itera sobre los datos, calcula la pérdida y ajusta los pesos.

3. **Implementación en PyTorch**
   - Usa `torch.nn.Module` para definir la red.
   - Usa `torch.utils.data.DataLoader` para manejar los datasets.
   - Define la función de pérdida y el optimizador.
   - Entrena la red y evalúa el desempeño.

4. **Experimentación**
   - Prueba diferentes arquitecturas: número de capas, neuronas, funciones de activación, tasas de aprendizaje, momentum, inicialización de pesos.
   - Compara los resultados y documenta cuál configuración funciona mejor.

5. **Comparación con el paper NoProp**
   - Lee el artículo y compara tus resultados con los reportados.
   - Si encuentras implementaciones públicas de NoProp, prueba y compara su desempeño.

6. **Informe**
   - Documenta el proceso, los experimentos y las conclusiones.
   - Incluye gráficos de precisión, pérdida, etc.

¿Te gustaría que te ayude a armar la estructura inicial del notebook o prefieres avanzar por tu cuenta y consultarme dudas puntuales?

### Implementación

In [5]:
class Perceptron:
    """
    Implementación de un perceptrón simple.
    Cada perceptrón tiene sus propios pesos y bias, y utiliza la función de activación sigmoide.
    """
    def __init__(self, input_size, learning_rate=0.1):
        # Inicialización de pesos y bias con valores pequeños aleatorios
        # Esto ayuda a romper la simetría y permite que cada neurona aprenda cosas distintas
        # self.weights = np.random.randn(input_size)
        # self.bias = np.random.randn()
        limit = np.sqrt(6 / (input_size + 1))
        self.weights = np.random.uniform(-limit, limit, input_size)
        self.bias = 0.0
        self.learning_rate = learning_rate

    def activation(self, x):
        # Función de activación sigmoide
        # Convierte la suma ponderada en un valor entre 0 y 1
        return 1 / (1 + np.exp(-x))

    def activation_derivative(self, x):
        # https://interactivechaos.com/es/manual/tutorial-de-deep-learning/derivada-de-la-funcion-sigmoide
        # Derivada de la sigmoide
        # x es el valor de la activación sigmoide. Es decir, se ejecutará esto en realidad => sigmoid(x) * (1 - sigmoid(x))
        return x * (1 - x)

    def forward(self, x):
        # Propagación hacia adelante de una sola muestra
        # Calcula la suma ponderada y aplica la función de activación
        z = np.dot(x, self.weights) + self.bias
        return self.activation(z)

    def update(self, x, delta):
        # Actualiza los pesos y bias usando el gradiente calculado (delta)
        # delta ya incluye la derivada de la sigmoide (por la regla de la cadena)
        # La actualización sigue la dirección del gradiente descendente
        self.weights += self.learning_rate * delta * x  # probar con -= capaz
        self.bias += self.learning_rate * delta # probar con -= capaz

In [ ]:
class NeuralNetwork:
    """
    Red neuronal multicapa compuesta por capas de perceptrones.
    Permite definir cualquier cantidad de capas y neuronas por capa.
    """
    def __init__(self, layer_sizes, learning_rate=0.1):
        # layer_sizes: lista con el tamaño de cada capa, EJEMPLO: [784, 100, 10]
        self.layers: list[list[Perceptron]] = []
        self.learning_rate = learning_rate
        for i in range(1, len(layer_sizes)):
            capa = [Perceptron(layer_sizes[i-1], learning_rate) for _ in range(layer_sizes[i])]
            # EJEMPLO:
            # - Para la primera capa oculta generaria 100 neuronas que aceptan vectores inputs de 784 dimensiones (input layer).
            # - Para la capa de input no se crea un perceptrón, ya que es la entrada de la red.
            # - La capa de salida generaria 10 neuronas que aceptan vectores inputs de 100 dimensiones (capa oculta).
            self.layers.append(capa)
        
        # Mostrar la estructura de la red
        # print("Estructura de la red:")
        # for i, capa in enumerate(self.layers):
        #     print(f"Capa {i+1}: {len(capa)} neuronas, cada una con {capa[0].weights.shape[0]} pesos")
        # print("\nPesos de la capa intermedia:")
        # for idx, neuron in enumerate(self.layers[0]):
        #     print(f"Neurona {idx}: {neuron.weights}")
        print("\nPesos de la capa de salida:")
        for idx, neuron in enumerate(self.layers[-1]):
            print(f"Neurona {idx}: {neuron.weights}")

    def forward(self, x):
        # Propagación hacia adelante para una muestra
        activations = [x]   # Inicialmente arranca con el input (x, vector de entrada)
        for capa in self.layers:
            salida_capa = np.array([neuron.forward(activations[-1]) for neuron in capa])
            activations.append(salida_capa)
        return activations

    def predict(self, x):
        # Predicción para una muestra
        activations = self.forward(x)
        return activations[-1]

    def train(self, X, Y, epochs=10):
        # Entrenamiento usando backpropagation (solo para fines didácticos, no optimizado)
        for epoch in range(epochs):
            for xi, yi in zip(X, Y):
                # Forward
                activations = self.forward(xi)  # activations es una lista de arrays, donde cada array es la salida de cada capa (función de activación sigmoidea aplicada)
                # Backward
                deltas = [None] * len(self.layers)
                # Capa de salida
                error = yi - activations[-1]    # Resta de vectores
                deltas[-1] = error * np.array([
                    neuron.activation_derivative(activations[-1][j])
                    for j, neuron in enumerate(self.layers[-1])
                ])
                # Capas ocultas
                for l in reversed(range(len(self.layers)-1)):
                    delta_next = deltas[l+1]    # lista de deltas de la capa siguiente (son 10 deltas)
                    pesos_next = np.array([neuron.weights for neuron in self.layers[l+1]])  # En la primera iteracion para MNIST, es un array de arrays de pesos de la capa siguiente -> matriz de 10 filas y 100 columnas
                    # deprecated: deltas[l] = self.layers[l][0].activation_derivative(activations[l+1]) * np.dot(pesos_next.T, delta_next)
                    deltas[l] = np.array([
                        neuron.activation_derivative(activations[l+1][j])
                        for j, neuron in enumerate(self.layers[l])
                    ]) * np.dot(pesos_next.T, delta_next)   # el resultado del producto escalar (np.dot) es un vector de 100 dimensiones (deltas de la capa oculta)

                # Actualización de pesos
                for l, capa in enumerate(self.layers):
                    for j, neuron in enumerate(capa):
                        neuron.update(activations[l], deltas[l][j])
                        
            if epoch % max(1, epochs//10) == 0:
                print(f"Epoch {epoch+1}/{epochs} completada")

### Prueba de dataset MNIST

In [213]:
# Preparamos un subconjunto pequeño de MNIST para entrenamiento rápido
# Usamos solo las primeras 100 muestras para entrenamiento y 10 para test
# X_train = x_train.reshape((x_train.shape[0], -1))[:1000]
# y_train = y_train[:1000]
# X_test = x_test.reshape((x_test.shape[0], -1))[:100]
# y_test = y_test[:100]

# 1. Seleccionamos un porcentaje del total (por ejemplo, 200 muestras)
n_samples = 10000  # Cambia este valor para usar más o menos datos
X = x_train.reshape((x_train.shape[0], -1))[:n_samples]
Y = y_train[:n_samples]

# 2. Hacemos un split 80/20 sobre ese subconjunto
split = int(0.8 * n_samples)
X_train = X[:split]
y_train = Y[:split]
X_test = X[split:]
y_test = Y[split:]

y_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [7]:
# Definimos la arquitectura: 1 capa oculta de 32 neuronas
input_size = X_train.shape[1]  # 784 para MNIST
hidden_size1 = 64
hidden_size2 = 32  # Capa oculta adicional
output_size = 10  # 10 clases

# Creamos la red
mlp = NeuralNetwork([input_size, hidden_size1, output_size], learning_rate=0.01)

# Entrenamos la red (puede tardar unos minutos)
mlp.train(X_train, y_train, epochs=100)


Pesos de la capa de salida:
Neurona 0: [ 0.25277594 -0.25901185 -0.23589625 -0.21621475  0.04147355 -0.19284623
  0.29746064  0.21780454  0.11796198  0.24362143  0.19985841 -0.13193108
  0.20957625  0.09096833  0.21613426 -0.08973384 -0.15433086 -0.13624272
 -0.27474413  0.23855722 -0.00573226  0.28681965 -0.23959232 -0.04457245
 -0.05709928 -0.22808522 -0.02219191  0.24869431  0.08852247 -0.29075464
 -0.16868892  0.03969206  0.07012052 -0.21624815  0.22912235 -0.09576322
  0.19148913 -0.07847247  0.22975221  0.13776957 -0.1418637   0.20953545
 -0.19184685 -0.02113817  0.01590189 -0.1263852   0.14541526  0.26099336
  0.27415673  0.07376886  0.23037205 -0.11027986  0.21199524 -0.13335125
 -0.13598982 -0.27195143 -0.14451862 -0.27361537  0.21877152  0.09996908
  0.01048936 -0.11595579  0.25824701  0.11299453]
Neurona 1: [ 0.07853277  0.06193662 -0.11827783  0.25410143 -0.21954503  0.25838437
  0.29308215 -0.02211279  0.11740586 -0.10005625  0.12220754  0.01493195
 -0.02497028  0.2284202

KeyboardInterrupt: 

In [215]:
# Evaluamos la red en el subconjunto de test
correct = 0
for xi, yi in zip(X_test, y_test):
    pred = mlp.predict(xi)
    if np.argmax(pred) == np.argmax(yi):
        correct += 1
print(f"Precisión en el subconjunto de test: {correct}/{len(X_test)}")

Precisión en el subconjunto de test: 1851/2000


### Prueba de dataset CIFAR-10

In [9]:
# Cargamos los datos
X_train, y_train, X_test, y_test = load_cifar10(f"data/CIFAR-10")

# Aplanamos las imágenes (de 32x32x3 a 3072)
X_train = X_train.reshape(-1, 32 * 32 * 3)
X_test = X_test.reshape(-1, 32 * 32 * 3)

# Definimos la arquitectura: dos capas ocultas
input_size = X_train.shape[1]  # 3072 para CIFAR-10
hidden_size1 = 64
hidden_size2 = 32
output_size = 10  # 10 clases para CIFAR-10

# Creamos la red con dos capas ocultas
mlp = NeuralNetwork([input_size, hidden_size1, hidden_size2, output_size], learning_rate=0.01)

# Entrenamos la red
mlp.train(X_train, y_train, epochs=100)

# (Opcional) Evaluar en el conjunto de prueba
mlp.evaluate(X_test, y_test)


Pesos de la capa de salida:
Neurona 0: [-0.21095034  0.21576708  0.29364339  0.1400519   0.41996989 -0.22393205
  0.2913735   0.23646594  0.26530075  0.18394691  0.41360217  0.15914489
  0.28223752  0.36638669 -0.39185923 -0.37938075 -0.20005329 -0.30621715
  0.05675134 -0.22032994 -0.21634552 -0.26723514 -0.30643823  0.13575966
 -0.33102104 -0.06599616 -0.3861617   0.20792701 -0.00717329 -0.28160243
  0.09158698 -0.103872  ]
Neurona 1: [-0.16258689 -0.31891716  0.25810096 -0.16217864  0.34511453  0.1320015
  0.40096686 -0.40238846 -0.39156597  0.30287841  0.17026449  0.19696797
  0.14788114  0.24497191  0.29448075  0.26878502 -0.16070508  0.35668421
 -0.0928961  -0.08774669  0.11814335 -0.24390586 -0.34729804  0.31618796
  0.11423405  0.09544808 -0.07302767 -0.11148791  0.22848843  0.31001171
  0.42480494  0.1443552 ]
Neurona 2: [ 0.37127601  0.4249363  -0.28742857  0.06093525 -0.01758693  0.10126086
 -0.23223909  0.40852539 -0.03028234 -0.3604767   0.31772827  0.28893932
 -0.2431276

KeyboardInterrupt: 

In [ ]:
# Evaluamos la red en el subconjunto de test
correct = 0
for xi, yi in zip(X_test, y_test):
    pred = mlp.predict(xi)
    if np.argmax(pred) == np.argmax(yi):
        correct += 1
print(f"Precisión en el subconjunto de test: {correct}/{len(X_test)}")

### Prueba de dataset CIFAR-100

In [ ]:
# Cargamos los datos
X_train, y_train, X_test, y_test = load_cifar100(f"data/CIFAR-100")

# Aplanamos las imágenes (de 32x32x3 a 3072)
X_train = X_train.reshape(-1, 32 * 32 * 3)
X_test = X_test.reshape(-1, 32 * 32 * 3)

# Definimos la arquitectura: dos capas ocultas
input_size = X_train.shape[1]  # 3072 para CIFAR-100
hidden_size1 = 64
hidden_size2 = 32
output_size = 100  # 100 clases para CIFAR-100

# Creamos la red con dos capas ocultas
mlp = NeuralNetwork([input_size, hidden_size1, hidden_size2, output_size], learning_rate=0.01)

# Entrenamos la red
mlp.train(X_train, y_train, epochs=100)

# (Opcional) Evaluar en el conjunto de prueba
mlp.evaluate(X_test, y_test)

# Implementación con Pytorch